In [33]:
## 现不依赖外部数据，由逻辑生成随机数据源进行推算
## 期望生成的 AI 先掌握由人类总结的逻辑
## 每种特征分几档，每档根据情况折算得分，得分高于 80 的认为最终可以上岗成功
## 后续业务数据逐步集成
## 特征之间可能相互影响，如家庭背景影响薪资权重，婚姻状况影响上班耗时权重（暂不考虑）
## 后续权重计算可能参考【折算成薪资后的数额进行】

In [34]:
import numpy as np
import pandas as pd

# 按照以下逻辑生成数据作为基础数据，表达目前的逻辑匹配推荐算法
SIZE = 2000
dict_raws = {
    # 职位类别 
    # 0 非常不满意
    # 1 不满意
    # 2 满意
    'job_categories': {
        'high': 3,
        'score': [0, 50, 100],
    },
    # 缴金
    # 0 公司缴纳+用户在乎
    # 1 公司缴纳+用户不在乎
    # 2 公司缴纳+用户在乎
    # 3 公司不缴纳+用户不在乎
    'insurance': {
        'high': 4,
        'score': [0, 50, 100, 100],
    },
    # 薪资+福利(包吃住等)
    # 0 薪资低于预期+无福利
    # 1 薪资低于预期+有福利
    # 2 薪资达到预期+无福利
    # 3 薪资达到预期+有福利
    'benefits': {
        'high': 4,
        'score': [0, 60, 80, 100],
    },
    # 上班耗时
    # 0 超过2小时
    # 1 超过1小时
    # 2 40分钟以内
    # 3 20分钟以内
    'go_work_time': {
        'high': 4,
        'score': [0, 20, 80, 100],
    },
}

df_raws = {}
for key in dict_raws:
    df_raws[key] = np.random.randint(dict_raws[key]['high'], size=(SIZE))
columns = dict_raws.keys()
df=pd.DataFrame(df_raws, columns=columns)

def get_sum_score(d):
    # 得分 0 总分 < 60 不考虑
    # 得分 1 总分 60 ~ 80 可以考虑
    # 得分 2 总分 > 80 有较强上岗可能性
    score = 0
    count = 0
    for key in columns:
        score = dict_raws[key]['score'][d[key]]
        # 暂时不设置权重，认为各特征权重相等 weight = dict_raws[key]['weight']
        count += (score / len(columns))
    if count < 60:
        score = 0
    if count >= 60 and count < 80:
        score = 1
    if count >= 80:
        score = 2
    return score

df["score"] = df.apply(get_sum_score,axis =1)

df

,job_categories,insurance,benefits,go_work_time,score
0,0,0,2,3,0
1,0,2,2,2,1
2,2,0,3,0,0
3,0,3,3,2,1
4,0,1,1,2,0
...,...,...,...,...,...
1995,2,0,2,2,1
1996,2,1,3,3,2
1997,1,1,2,0,0
1998,1,0,0,2,0


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

wine_data = df[columns].values
wine_target = df["score"].values
print(wine_data)
print(wine_target)


[[0 0 2 3]
 [0 2 2 2]
 [2 0 3 0]
 ...
 [1 1 2 0]
 [1 0 0 2]
 [2 0 3 0]]
[0 1 0 ... 0 0 0]


In [37]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(wine_data,wine_target,test_size=0.3)

r_lf = RandomForestClassifier(random_state=0)
r_lf = r_lf.fit(Xtrain,Ytrain)
r_score = r_lf.score(Xtest,Ytest)

print("Random Forest:{}".format(r_score))

Random Forest:1.0


In [38]:
print(r_lf.predict([[2,3,3,3]]))

[2]
